In [ ]:

import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Bidirectional,TimeDistributed
from keras.layers.embeddings import Embedding


from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('dataset.csv')
data= data.dropna()

In [ ]:
def preprocess(text):  
    text = text.translate(string.punctuation)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower()) #lower caseing
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) # removing special chars
data['text'] = data.text.str.replace('rt','') # removing "rt"

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
# print(X[:2])

Y = pd.get_dummies(data['class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)

(11972, 504) (11972, 2)


In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 504, 128)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 504, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 1)

Epoch 1/3
94/94 [==============================] - 617s 7s/step - loss: 0.6279 - accuracy: 0.6596
Epoch 2/3
94/94 [==============================] - 611s 6s/step - loss: 0.2663 - accuracy: 0.9047
Epoch 3/3
94/94 [==============================] - 611s 7s/step - loss: 0.1743 - accuracy: 0.9411


In [ ]:
data_majority = data[data['class'] == 0]
data_minority = data[data['class'] == 1]

bias = data_minority.shape[0]/data_majority.shape[0]

train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

print('positive data in training:',(train['class'] == 1).sum())
print('negative data in training:',(train['class'] == 0).sum())
print('positive data in test:',(test['class'] == 1).sum())
print('negative data in test:',(test['class'] == 0).sum())

positive data in training: 5635
negative data in training: 6337
positive data in test: 1409
negative data in test: 1584


Upsampling data

In [ ]:
data_majority = train[train['class'] == 0]
data_minority = train[train['class'] == 1]

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled['class'].value_counts(),sep = "")

majority class before upsample: (6337, 2)
minority class before upsample: (5635, 2)
After upsampling
1.0    6337
0.0    6337
Name: class, dtype: int64


In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['text'].values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['class']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['class']).values
print("x_test shape", X_test.shape)
embed_dim = 128
lstm_out = 192

x_train shape: (12674, 29)
x_test shape (2993, 29)


LSTM


In [ ]:

# model
model1 = Sequential()
model1.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model1.add(SpatialDropout1D(0.4))
model1.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model1.add(Dense(2,activation='softmax'))
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 29, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 192)               246528    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 386       
Total params: 502,914
Trainable params: 502,914
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1.6/bias }
model1.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/15
100/100 [==============================] - 37s 336ms/step - loss: 0.8014 - accuracy: 0.5984
Epoch 2/15
100/100 [==============================] - 34s 337ms/step - loss: 0.3541 - accuracy: 0.8971
Epoch 3/15
100/100 [==============================] - 34s 336ms/step - loss: 0.2504 - accuracy: 0.9271
Epoch 4/15
100/100 [==============================] - 33s 335ms/step - loss: 0.2498 - accuracy: 0.9348
Epoch 5/15
100/100 [==============================] - 33s 334ms/step - loss: 0.2464 - accuracy: 0.9233
Epoch 6/15
100/100 [==============================] - 34s 335ms/step - loss: 0.1846 - accuracy: 0.9509
Epoch 7/15
100/100 [==============================] - 33s 334ms/step - loss: 0.1596 - accuracy: 0.9573
Epoch 8/15
100/100 [==============================] - 33s 335ms/step - loss: 0.1570 - accuracy: 0.9588
Epoch 9/15
100/100 [==============================] - 33s 334ms/step - loss: 0.1385 - accuracy: 0.9647
Epoch 10/15
100/100 [==============================] - 34s 336ms/step - l

In [ ]:
Y_pred = model1.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print(classification_report(df_test.true, df_test.pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1584
           1       0.90      0.87      0.88      1409

    accuracy                           0.89      2993
   macro avg       0.89      0.89      0.89      2993
weighted avg       0.89      0.89      0.89      2993



BiLSTM

In [ ]:
model2 = Sequential()
model2.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model2.add(SpatialDropout1D(0.4))
model2.add(Bidirectional(LSTM(100, dropout=0.4, recurrent_dropout=0.4)))
model2.add(Dense(2,activation='softmax'))
model2.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 29, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               183200    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 439,602
Trainable params: 439,602
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 128
class_weights = {0: 1 ,
                1: 1.6/bias }
model2.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/15
100/100 [==============================] - 34s 284ms/step - loss: 0.8437 - accuracy: 0.5514
Epoch 2/15
100/100 [==============================] - 28s 284ms/step - loss: 0.3530 - accuracy: 0.8942
Epoch 3/15
100/100 [==============================] - 29s 286ms/step - loss: 0.2703 - accuracy: 0.9237
Epoch 4/15
100/100 [==============================] - 29s 287ms/step - loss: 0.2096 - accuracy: 0.9477
Epoch 5/15
100/100 [==============================] - 29s 287ms/step - loss: 0.1809 - accuracy: 0.9560
Epoch 6/15
100/100 [==============================] - 29s 286ms/step - loss: 0.1728 - accuracy: 0.9559
Epoch 7/15
100/100 [==============================] - 29s 288ms/step - loss: 0.1554 - accuracy: 0.9635
Epoch 8/15
100/100 [==============================] - 29s 286ms/step - loss: 0.1352 - accuracy: 0.9643
Epoch 9/15
100/100 [==============================] - 29s 287ms/step - loss: 0.1201 - accuracy: 0.9684
Epoch 10/15
100/100 [==============================] - 29s 287ms/step - l

In [ ]:
Y_pred = model2.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print(classification_report(df_test.true, df_test.pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1584
           1       0.90      0.88      0.89      1409

    accuracy                           0.89      2993
   macro avg       0.89      0.89      0.89      2993
weighted avg       0.89      0.89      0.89      2993



In [ ]:
model2.evaluate(X_test,Y_test)

94/94 [==============================] - 2s 19ms/step - loss: 0.4315 - accuracy: 0.8938


[0.4314849376678467, 0.8937520980834961]

Save model


In [ ]:
model2.save('bilstm_final.h5') 

In [ ]:
new_model = tf.keras.models.load_model('bilstm_final.h5')
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 29, 128)           256000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 29, 128)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               183200    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 402       
Total params: 439,602
Trainable params: 439,602
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, acc = new_model.evaluate(X_test, Y_test, verbose=1)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

94/94 [==============================] - 3s 20ms/step - loss: 0.4315 - accuracy: 0.8938
Restored model, accuracy: 89.38%


Predict

In [ ]:
!pip install google_play_scraper

     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-0.2.1-cp37-none-any.whl size=22199 sha256=141cd226dc069aa5fb2b81e54164c943912189dd6b0ccee217b62dd34f1d0148
  Stored in directory: /root/.cache/pip/wheels/9a/fe/59/aaed9c7885041da68908f5e45f2796bf77696ac83874afd02c
Successfully built google-play-scraper


In [ ]:
from google_play_scraper import app
import re
import nltk
from nltk.tokenize import word_tokenize
import csv

In [ ]:
link="https://play.google.com/store/apps/details?id=com.meesho.supply"
findId=link.find('id=')

url=link[findId+3:]
file = open("cc.txt", "w",encoding='utf-8')
file.write(str(app(
    url,
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
)))
file.close()

In [ ]:
myfile=[]
with open("cc.txt",encoding='utf8') as mydata:
	for data in mydata:
		myfile.append(data)
start=myfile[0].find('comments')
end=myfile[0].find('editorsChoice')
c=data[start:end]

In [ ]:
def preprocess(text):  
    text = text.translate(string.punctuation)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [ ]:
c= c.lower() 
c =  re.sub('[^a-zA-z0-9\s]','',c) 
c= c.replace('rt','') 

In [ ]:
nltk.download('stopwords')
c = preprocess(c)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
tokenizer = Tokenizer(num_words= 200)
tokenizer.fit_on_texts(c)

sequences = tokenizer.texts_to_sequences(c)
data = pad_sequences(sequences, maxlen=200)
result=np.argmax(new_model.predict(data), axis=-1)
result

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
result.sum()/len(result)

0.00995732574679943